<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Market-Data/blob/main/Order_Book_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GET /openApi/swap/v2/quote/depth
# 不需要 API 金鑰簽名

In [ ]:
import time
import requests
import hmac
from hashlib import sha256

In [ ]:
APIURL = "https://open-api.bingx.com"  # API 基礎網址
APIKEY = ""  # API 金鑰，需填入使用者的有效金鑰
SECRETKEY = ""  # 秘密金鑰，需填入使用者的密鑰

In [ ]:
def demo():
    payload = {}  # GET 請求通常無額外請求體
    path = '/openApi/swap/v2/quote/depth'  # API 路徑：獲取訂單簿深度
    method = "GET"  # HTTP 請求方法


    paramsMap = {
        "symbol": "SHIB-USDT",  # 交易對，範例為 SHIB 對 USDT
        "limit": "5"  # 限制返回的訂單數量，這裡取 5 條
    }

    paramsStr = parseParam(paramsMap)  # 將參數字典轉換成 URL 查詢字串並加入時間戳
    return send_request(method, path, paramsStr, payload)  # 發送 API 請求並回傳結果

In [ ]:
def get_sign(api_secret, payload):
    # 用 HMAC-SHA256 生成簽名，payload 是已排序並組合成字串的參數

    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 印出簽名供除錯用
    return signature  # 回傳簽名字串

In [ ]:
def send_request(method, path, urlpa, payload):
    # 組合完整請求 URL，包含簽名參數
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 印出完整 URL 方便除錯

    headers = {
        'X-BX-APIKEY': APIKEY,  # HTTP 標頭帶入 API 金鑰
    }

    # 發送 HTTP 請求，method、URL、標頭與請求體
    response = requests.request(method, url, headers=headers, data=payload)
    return response.text  # 回傳 API 回應內容（文字）

In [ ]:
def parseParam(paramsMap):
    sortedKeys = sorted(paramsMap)  # 參數鍵排序，確保簽名一致

    # 參數轉成 URL 查詢字串 key=value&key2=value2
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])

    if paramsStr != "":
        # 如果有其他參數，加上 timestamp 時間戳（毫秒）
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        # 若無參數，只有 timestamp
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
if __name__ == '__main__':
    print("demo:", demo())  # 執行範例並印出結果